In [18]:
import os, sys, subprocess, re, requests, json, warnings
import numpy as np
import pandas as pd
from urllib.request import urlopen
from time import sleep
from Bio import SeqIO

warnings.filterwarnings('ignore')

In [11]:
def get_url(url, **kwargs):
    '''
    Obatin a response from a given url
    '''
    response = requests.get(url, **kwargs);

    if not response.ok:
        print(response.text)
        response.raise_for_status()
        sys.exit()

    return response

In [12]:
# uniprot API URL
WEBSITE_API = "https://rest.uniprot.org/uniprotkb/"

# 1. FASTA sequence

In [7]:
# protein list import
df = pd.read_excel('./SourceData/protein_list.xlsx')
columns = df.iloc[0, 1:]
df = df.iloc[1:, 1:]
df.columns = columns

In [ ]:
# Protein entry list
entry_list = df.Entry.unique().tolist()

# AH location start postion list
AH_start_list = df.AH_location_start.unique

In [24]:
# Obtain FASTA seq from Uniprot, make AH position data
for entry in df.Entry:
    print(entry)
    try:
        r = get_url(f'{WEBSITE_API}search?query=accession:{entry}&format=fasta&compressed=false')
    except requests.exceptions.ConnectionError:
        r.status_code = "Connection refused"
        break
        
    # export fasta
    fasta = r.text
    with open('./TrainingData/Original_fasta/%s.fas' % entry, 'w') as f:
        f.write(fasta)
        f.close()
        
    sleep(1)

Q9UH99
Q8N6T3
Q15643


# Secondary structure prediction by s4pred

In [26]:
model = '../s4pred/run_model.py'
for entry in df.Entry:
    command = \
    'python %s --outfmt fas ./TrainingData/Original_fasta/%s.fas > ./TrainingData/s4pred_output/%s_ss.fas' \
    % (model, entry, entry)
    subprocess.run(command, shell=True)
    print(entry, ' done')

Q9UH99  done
Q8N6T3  done
Q15643  done


In [47]:
file = open('./TrainingData/s4pred_output/Q15643_ss.fas', 'r')
iterator = SeqIO.read(file, 'fasta')
# records = SeqIO.to_dict(iterator)
print(iterator.seq)
# print(list)
#     print(seq_rec.seq)
file.close()


# lines = file.readlines()
# lines[1:]
# seq = ''.join(lines[1:])

MSSWLGGLGSGLGQSLGQVGGSLASLTGQISNFTKDMLMEGTEEVEAELPDSRTKEIEAIHAILRSENERLKKLCTDLEEKHEASEIQIKQQSTSYRNQLQQKEVEISHLKARQIALQDQLLKLQSAAQSVPSGAGVPATTASSSFAYGISHHPSAFHDDDMDFGDIISSQQEINRLSNEVSRLESEVGHWRHIAQTSKAQGTDNSDQSEICKLQNIIKELKQNRSQEIDDHQHEMSVLQNAHQQKLTEISRRHREELSDYEERIEELENLLQQGGSGVIETDLSKIYEMQKTIQVLQIEKVESTKKMEQLEDKIKDINKKLSSAENDRDILRREQEQLNVEKRQIMEECENLKLECSKLQPSAVKQSDTMTEKERILAQSASVEEVFRLQQALSDAENEIMRLSSLNQDNSLAEDNLKLKMRIEVLEKEKSLLSQEKEELQMSLLKLNNEYEVIKSTATRDISLDSELHDLRLNLEAKEQELNQSISEKETLIAEIEELDRQNQEATKHMILIKDQLSKQQNEGDSIISKLKQDLNDEKKRVHQLEDDKMDITKELDVQKEKLIQSEVALNDLHLTKQKLEDKVENLVDQLNKSQESNVSIQKENLELKEHIRQNEEELSRIRNELMQSLNQDSNSNFKDTLLKEREAEVRNLKQNLSELEQLNENLKKVAFDVKMENEKLVLACEDVRHQLEECLAGNNQLSLEKNTIVETLKMEKGEIEAELCWAKKRLLEEANKYEKTIEELSNARNLNTSALQLEHEHLIKLNQKKDMEIAELKKNIEQMDTDHKETKDVLSSSLEEQKQLTQLINKKEIFIEKLKERSSKLQEELDKYSQALRKNEILRQTIEEKDRSLGSMKEENNHLQEELERLREEQSRTAPVADPKTLDSVTELASEVSQLNTIKEHLEEEIKHHQKIIEDQNQSKMQLLQSLQEQKKEMDEFRYQHEQMNATHTQLFLEKDEEIKSLQKTIEQIKTQLHEERQDIQTDNSDIFQETKVQ

In [36]:
seq = seq.replace('\n', '')
len(seq)

1979

In [48]:
import biolib

In [49]:
deeptmhmm = biolib.load('DTU/DeepTMHMM')

2022-07-31 21:59:36,489 | INFO : Loaded project DTU/DeepTMHMM:1.0.12


In [50]:
# Run DeepTMHMM
deeptmhmm_res = deeptmhmm.cli(args='--fasta ./TrainingData/Original_fasta/Q15643.fas')

2022-07-31 22:00:07,687 | INFO : Job "f56440db-8641-4156-a095-17a27917bd73" is starting...
2022-07-31 22:00:28,748 | INFO : Cloud: Server capacity is being allocated. Please wait...
2022-07-31 22:00:39,328 | INFO : Cloud: Server capacity is being allocated. Please wait...
2022-07-31 22:00:49,620 | INFO : Cloud: Server capacity is being allocated. Please wait...
2022-07-31 22:00:59,906 | INFO : Cloud: Server capacity is being allocated. Please wait...
2022-07-31 22:01:10,173 | INFO : Cloud: Server capacity is being allocated. Please wait...
2022-07-31 22:01:20,431 | INFO : Cloud: Server capacity is being allocated. Please wait...
2022-07-31 22:01:30,670 | INFO : Cloud: Server capacity is being allocated. Please wait...
2022-07-31 22:01:40,949 | INFO : Cloud: Server capacity is being allocated. Please wait...
2022-07-31 22:01:51,181 | INFO : Cloud: Server capacity is being allocated. Please wait...
2022-07-31 22:02:01,428 | INFO : Cloud: Server capacity is being allocated. Please wait...

In [51]:
# Save the results
deeptmhmm_res.save_files("./biolib_results/")

In [52]:
deeptmhmm_res.ipython_markdown()

## DeepTMHMM - Predictions
Predicted topologies can be downloaded in [.gff3 format](TMRs.gff3) and [.3line format](predicted_topologies.3line)
![picture](././biolib_results//plot.png)
You can download the probabilities used to generate this plot [here](sp|Q15643|TRIPB_HUMAN_probs.csv)
### Predicted Topologies
```
>sp|Q15643|TRIPB_HUMAN | GLOB
MSSWLGGLGSGLGQSLGQVGGSLASLTGQISNFTKDMLMEGTEEVEAELPDSRTKEIEAIHAILRSENERLKKLCTDLEEKHEASEIQIKQQSTSYRNQLQQKEVEISHLKARQIALQDQLLKLQSAAQSVPSGAGVPATTASSSFAYGISHHPSAFHDDDMDFGDIISSQQEINRLSNEVSRLESEVGHWRHIAQTSKAQGTDNSDQSEICKLQNIIKELKQNRSQEIDDHQHEMSVLQNAHQQKLTEISRRHREELSDYEERIEELENLLQQGGSGVIETDLSKIYEMQKTIQVLQIEKVESTKKMEQLEDKIKDINKKLSSAENDRDILRREQEQLNVEKRQIMEECENLKLECSKLQPSAVKQSDTMTEKERILAQSASVEEVFRLQQALSDAENEIMRLSSLNQDNSLAEDNLKLKMRIEVLEKEKSLLSQEKEELQMSLLKLNNEYEVIKSTATRDISLDSELHDLRLNLEAKEQELNQSISEKETLIAEIEELDRQNQEATKHMILIKDQLSKQQNEGDSIISKLKQDLNDEKKRVHQLEDDKMDITKELDVQKEKLIQSEVALNDLHLTKQKLEDKVENLVDQLNKSQESNVSIQKENLELKEHIRQNEEELSRIRNELMQSLNQDSNSNFKDTLLKEREAEVRNLKQNLSELEQLNENLKKVAFDVKMENEKLVLACEDVRHQLEECLAGNNQLSLEKNTIVETLKMEKGEIEAELCWAKKRLLEEANKYEKTIEELSNARNLNTSALQLEHEHLIKLNQKKDMEIAELKKNIEQMDTDHKETKDVLSSSLEEQKQLTQLINKKEIFIEKLKERSSKLQEELDKYSQALRKNEILRQTIEEKDRSLGSMKEENNHLQEELERLREEQSRTAPVADPKTLDSVTELASEVSQLNTIKEHLEEEIKHHQKIIEDQNQSKMQLLQSLQEQKKEMDEFRYQHEQMNATHTQLFLEKDEEIKSLQKTIEQIKTQLHEERQDIQTDNSDIFQETKVQSLNIENGSEKHDLSKAETERLVKGIKERELEIKLLNEKNISLTKQIDQLSKDEVGKLTQIIQQKDLEIQALHARISSTSHTQDVVYLQQQLQAYAMEREKVFAVLNEKTRENSHLKTEYHKMMDIVAAKEAALIKLQDENKKLSTRFESSGQDMFRETIQNLSRIIREKDIEIDALSQKCQTLLAVLQTSSTGNEAGGVNSNQFEELLQERDKLKQQVKKMEEWKQQVMTTVQNMQHESAQLQEELHQLQAQVLVDSDNNSKLQVDYTGLIQSYEQNETKLKNFGQELAQVQHSIGQLCNTKDLLLGKLDIISPQLSSASLLTPQSAECLRASKSEVLSESSELLQQELEELRKSLQEKDATIRTLQENNHRLSDSIAATSELERKEHEQTDSEIKQLKEKQDVLQKLLKEKDLLIKAKSDQLLSSNENFTNKVNENELLRQAVTNLKERILILEMDIGKLKGENEKIVETYRGKETEYQALQETNMKFSMMLREKEFECHSMKEKALAFEQLLKEKEQGKTGELNQLLNAVKSMQEKTVVFQQERDQVMLALKQKQMENTALQNEVQRLRDKEFRSNQELERLRNHLLESEDSYTREALAAEDREAKLRKKVTVLEEKLVSSSNAMENASHQASVQVESLQEQLNVVSKQRDETALQLSVSQEQVKQYALSLANLQMVLEHFQQEEKAMYSAELEKQKQLIAEWKKNAENLEGKVISLQECLDEANAALDSASRLTEQLDVKEEQIEELKRQNELRQEMLDDVQKKLMSLANSSEGKVDKVLMRNLFIGHFHTPKNQRHEVLRLMGSILGVRREEMEQLFHDDQGGVTRWMTGWLGGGSKSVPNTPLRPNQQSVVNSSFSELFVKFLETESHPSIPPPKLSVHDMKPLDSPGRRKRDTNAPESFKDTAESRSGRRTDVNPFLAPRSAAVPLINPAGLGPGGPGHLLLKPISDVLPTFTPLPALPDNSAGVVLKDLLKQ
IIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII

```


```
##gff-version 3
# sp|Q15643|TRIPB_HUMAN Length: 1979
# sp|Q15643|TRIPB_HUMAN Number of predicted TMRs: 0
sp|Q15643|TRIPB_HUMAN	inside	1	1979				

```
